In [119]:
import sys 
sys.path.append('SGG_Benchmark_main')
from SGG_Benchmark_main.demo.demo_model import SGG_Model
import cv2
import os 
import numpy as np

def get_pre(source):

    config_path = "SGG_Benchmark_main\\VG150-penet-yolov8m\\config.yml"
    dict_path = "SGG_Benchmark_main\\datasets\\vg\\VG-SGG-dicts-with-attri.json"
    weights_path = "SGG_Benchmark_main\\VG150-penet-yolov8m\\best_model_epoch_2.pth"

    example_img = source  

    img = cv2.imread(example_img)
    #img=cv2.resize(img,(1024,1024))

    model = SGG_Model(config_path, dict_path, weights_path, rel_conf=0.01, box_conf=0.1, show_fps=False)

    predictions = model.predict(img)

    return predictions,img.shape[:2]

In [122]:
def get_box_pos(bbox,orig_size):
    """
    inputs:
        bbox 框的位置 xywh
        orig_size 图像大小 x,y heng shu
    outputs:
        框所在图片位置
        左上角，上部，右上角
        左部，  中部，右部
        左下角  下部  右下角
        012
        345
        678
           
    """
    center=((bbox[0]+bbox[2])/2,(bbox[1]+bbox[3])/2)    
    #center=(bbox[0],bbox[1])

    if center[0]<orig_size[0]/3 and center[1]<orig_size[1]/3:
        return 'top left corner'
    if center[0]>=orig_size[0]/3 and center[0]<=2*orig_size[0]/3 and center[1]<orig_size[1]/3:
        return 'top'
    if center[0]>2*orig_size[0]/3 and center[1]<orig_size[1]/3:
        return 'top right corner'
    if center[0]<orig_size[0]/3 and center[1]>=orig_size[1]/3 and center[1]<=2*orig_size[1]/3:
        return 'left'
    if center[0]>=orig_size[0]/3 and center[0]<=2*orig_size[0]/3 and center[1]>=orig_size[1]/3 and center[1]<=2*orig_size[1]/3:
        return 'center'
    if center[0]>2*orig_size[0]/3 and center[1]>=orig_size[1]/3 and center[1]<=2*orig_size[1]/3:
        return 'right'
    if center[0]<orig_size[0]/3 and center[1]>2*orig_size[1]/3:
        return 'lower left corner'
    if center[0]>=orig_size[0]/3 and center[0]<=2*orig_size[0]/3 and center[1]>2*orig_size[1]/3:
        return 'lower'
    if center[0]>2*orig_size[0]/3 and center[1]>2*orig_size[1]/3:
        return 'lower right corner'



def deal_predictions(predictions):
    pos_obj_rel_pos_sub=[]         #


    for i in range(len(predictions['rel_labels'])):
        obj_index,sub_index=predictions['rel_pairs'][i][0],predictions['rel_pairs'][i][1]

        obj,sub=predictions['bbox_labels'][obj_index],predictions['bbox_labels'][sub_index]
        obj_bbox,sub_bbox=predictions['bbox'][obj_index],predictions['bbox'][sub_index]
        rel=predictions['rel_labels'][i]

        pos_obj_rel_pos_sub.append([get_box_pos(obj_bbox),obj,rel,get_box_pos(sub_bbox),sub])

    object_count={}
    for i in predictions['bbox_labels']:
        if i in object_count.keys():
            object_count[i]+=1
        else:
            object_count[i]=1
    
    
    return pos_obj_rel_pos_sub,object_count

def fix(predictions,probs=0.15):
    fix=np.array(predictions['bbox_scores'])>probs

    bbox=np.array(predictions['bbox'])[fix]
    bbox_labels=np.array(predictions['bbox_labels'])[fix]

    bbox_idx=bbox_labels.shape[0]

    rel_pairs=[]
    rel_labels=[]

    for i,pair in enumerate(predictions['rel_pairs']):
        if pair[0]<bbox_idx and pair[1]<bbox_idx:
            rel_pairs.append(pair)
            rel_labels.append(predictions['rel_labels'][i])

    return list(bbox),list(bbox_labels),rel_pairs,rel_labels

def deal_SGG_result(predictions,shape):


    bbox,bbox_labels,rel_pairs,rel_labels=fix(predictions)

    
    object_count={}
    #for i in predictions['bbox_labels']:
    for i in bbox_labels:
        if i in object_count.keys():
            object_count[i]+=1
        else:
            object_count[i]=1

    location_dict={key:[] for key in object_count.keys()}

    #for i in range(len(predictions['bbox'])):
    for i in range(len(bbox)):
        #location_dict[predictions['bbox_labels'][i]].append(get_box_pos(predictions['bbox'][i],shape))
        location_dict[bbox_labels[i]].append(get_box_pos(bbox[i],shape))


    relation_dict=[]

    #for i in range(len(predictions['rel_pairs'])):
    #    relation_dict.append([predictions['bbox_labels'][predictions['rel_pairs'][i][0]],predictions['rel_labels'][i],predictions['bbox_labels'][predictions['rel_pairs'][i][1]]])
    for i in range(len(rel_pairs)):
        relation_dict.append([bbox_labels[rel_pairs[i][0]],rel_labels[i],bbox_labels[rel_pairs[i][1]]])

    return {"number":object_count,"location":location_dict,"relationship":relation_dict}




    

In [4]:
from openai import OpenAI
import json
import os


In [10]:
def get_response(pre):
    client = OpenAI(
        api_key="sk-34272c378ff946f48c7e24ec38dc6030", #
        base_url="Your keys",  
    )

    Question='Please complete the following task based on the data {} (which contains four parts of information: quantity,location,and relationship):'.format(json.dumps(pre))+'{"Task": "Please provide me with the category,quantity,position,and relationship of all different objects in the given image in JSON format,only output json\
    For quantity, count all the quantities of different objects. If it is 0, do not output that class,\
    For the positions of different objects, the position of each individual unit in each type of object needs to be output(the positions are divided into nine grids, namely the top left corner,\
    top,top right corner,left,center,right,left lower corner,lower,right lower corner)。\
    For relationships, output the relationships between all objects in the image in the form of triplets, [object1, relation,object2],\
    Please refer to the following JSON format for specific formatting, and use this JSON format for output (just output one JSON)",\
    "Restriction": "Strictly provide JSON responses in <format> format,Do not output any other information and irrelevant symbols,Please provide me with a string format and do not output the syntax for markdown,Please check your output and do not output irrelevant symbols",\
    "format":{"category":{"man","car","woman"},"number":{"man":2,"car":2,"woman":1},"location":{"man":{1:"center",2:"lower"},"car":{1,:"top right corner",2:"left"},"woman":{1:"center"} },"relationship":[["man","near to","car"],["car","on","load"],["woman","has","car"]]} }'


    completion = client.chat.completions.create(
        model="qwen-plus-0806",
        messages=[
            {'role': 'user', 'content': Question}],
        seed=2
        )
    return completion

In [143]:
pre_output_list=[]

list_dir=os.listdir('VG_data\\test_data')

import pickle 

for i in list_dir[-200:-100]:

    predictions,shape=get_pre(os.path.join('VG_data\\test_data',i))

    pre=deal_SGG_result(predictions,shape)

    #pre_output_list.append(pre)

    with open(os.path.join('VG_pre/'+i[:-4]+'.pickle'),'wb') as f:
        pickle.dump(pre,f)
    

2024-09-02 19:28:16.348 | INFO     | sgg_benchmark.utils.logger:setup_logger:31 - Using loguru logger with level: INFO
Overriding model.yaml nc=80 with nc=150
loading word vectors from /home/maelic/glove\glove.6B.200d.pt
loading word vectors from /home/maelic/glove\glove.6B.200d.pt
2024-09-02 19:28:22.509 | INFO     | sgg_benchmark.utils.logger:setup_logger:31 - Using loguru logger with level: INFO
Overriding model.yaml nc=80 with nc=150
loading word vectors from /home/maelic/glove\glove.6B.200d.pt
loading word vectors from /home/maelic/glove\glove.6B.200d.pt
2024-09-02 19:28:27.781 | INFO     | sgg_benchmark.utils.logger:setup_logger:31 - Using loguru logger with level: INFO
Overriding model.yaml nc=80 with nc=150
loading word vectors from /home/maelic/glove\glove.6B.200d.pt
loading word vectors from /home/maelic/glove\glove.6B.200d.pt
2024-09-02 19:28:33.455 | INFO     | sgg_benchmark.utils.logger:setup_logger:31 - Using loguru logger with level: INFO
Overriding model.yaml nc=80 with

NameError: name 'predictions' is not defined

In [12]:
with open('VG_pre/2339076.pickle','rb') as f:
    pre=pickle.load(f)


In [13]:
output=get_response(pre)
out=output.choices[0].message.content

In [14]:
out

'{"number":{"pant":1,"helmet":2,"man":3,"shirt":2,"shoe":2,"glove":3,"player":1,"head":2,"number":1,"seat":1,"stand":1},"location":{"pant":{1:"right"},"helmet":{1:"top",2:"top right corner"},"man":{1:"right",2:"right",3:"top right corner"},"shirt":{1:"top",2:"right"},"shoe":{1:"lower right corner",2:"lower"},"glove":{1:"top",2:"top",3:"right"},"player":{1:"right"},"head":{1:"top",2:"top right corner"},"number":{1:"top"},"seat":{1:"top"},"stand":{1:"top"}},"relationship":[["shirt","on","man"],["shirt","on","man"],["head","on","man"],["head","on","shirt"],["helmet","on","man"],["helmet","on","player"],["head","on","man"],["helmet","on","man"],["pant","on","man"],["shirt","on","player"],["shirt","near","man"],["pant","near","player"],["glove","on","shirt"],["helmet","on","head"],["helmet","on","shirt"],["glove","on","man"],["glove","on","man"],["head","on","shirt"],["head","on","player"],["glove","on","shirt"],["number","on","man"],["glove","on","man"],["shoe","near","man"],["shoe","near"

In [300]:
with open(os.path.join('my/596.pickle'),'wb') as f:
    pickle.dump(out,f)

In [2]:
import demjson3
import os

In [11]:
list_dir=os.listdir('VG_pre')

import pickle 


for j,i in enumerate(list_dir[-100:]):

    if j<19:
        continue


    with open('VG_pre/'+i,'rb') as f:
        pre=pickle.load(f)


    output=get_response(pre)

    out=output.choices[0].message.content

    predict=demjson3.decode(out)

    with open(os.path.join('VG_my',i),'wb') as f:
        pickle.dump(out,f)

    print(i)

JSONDecodeError: Unexpected text after end of JSON value

In [12]:
out

'{"number":{"elephant":1,"leaf":5,"man":1,"tree":1,"shirt":1},"location":{"elephant":{"1":"right"},"leaf":{"1":"top left corner","2":"top","3":"top left corner","4":"top left corner","5":"center"},"man":{"1":"right"},"tree":{"1":"right"},"shirt":{"1":"right"}},"relationship":[["man","on","elephant"],["leaf","near","tree"],["leaf","near","tree"],["leaf","near","tree"],["leaf","near","tree"],["shirt","on","tree"],["leaf","on","leaf"],["man","near","tree"],["elephant","near","tree"],["leaf","near","tree"],["shirt","near","elephant"],["shirt","near","man"],["tree","near","elephant"],["tree","has","leaf"],["tree","has","leaf"]]}"'

In [1]:
import pickle
import os 
import nltk
import demjson3
from nltk.corpus import wordnet
import re

In [2]:
def are_synonyms_split(word_list,word2):
    """第一个是一个列表"""
    right=[]

    for i in word_list:
        for j in word2:
            right.append(are_synonyms(i,j))

    if True in right:
        return True 
    else:
        return False

def has_hypernym_or_hyponym_split(word_list,word2):
    """第一个是一个列表"""
    right=[]

    for i in word_list:
        for j in word2:
            right.append(has_hypernym_or_hyponym(i,j))

    if True in right:
        return True 
    else:
        return False

def are_synonyms(word1, word2):
    # 获取word1的所有同义词集
    synsets1 = wordnet.synsets(word1)
    
    # 获取word2的所有同义词集
    synsets2 = wordnet.synsets(word2)

    
    
    # 检查是否有交集
    for synset1 in synsets1:
        for synset2 in synsets2:
            if synset1 == synset2:
                return True
    
    # 计算路径相似度作为备选方案
    for synset1 in synsets1:
        for synset2 in synsets2:
            if synset1.path_similarity(synset2) is not None and synset1.path_similarity(synset2) > 0.8:
                return True
    
    return False

def has_hypernym_or_hyponym(word1, word2):
    # 获取word1的所有同义词集
    synsets1 = wordnet.synsets(word1)
    
    # 获取word2的所有同义词集
    synsets2 = wordnet.synsets(word2)
    
    # 检查是否存在上位词或下位词关系
    for synset1 in synsets1:
        for synset2 in synsets2:
            # 检查synset1是否是synset2的上位词
            if synset1 in synset2.hypernyms():
                return True
            
            # 检查synset2是否是synset1的上位词
            if synset2 in synset1.hypernyms():
                return True
    
    return False

def count_common_elements_with_repetition(list1, list2):
    # 创建字典来存储每个元素在两个列表中的出现次数
    count_dict1 = {}
    count_dict2 = {}
    
    # 统计第一个列表中每个元素的出现次数
    for element in list1:
        if element in count_dict1:
            count_dict1[element] += 1
        else:
            count_dict1[element] = 1
    
    # 统计第二个列表中每个元素的出现次数
    for element in list2:
        if element in count_dict2:
            count_dict2[element] += 1
        else:
            count_dict2[element] = 1
    
    # 计算共同元素的数量
    common_count = 0
    for element in count_dict1:
        if element in count_dict2:
            # 取两个列表中该元素出现次数的较小值
            common_count += min(count_dict1[element], count_dict2[element])
    
    return common_count

In [3]:
def compute_right_single(ground_truth,predict):
    """
        计算单张图片的预测准确率
        分成四个 计数，位置，关系，total
    """
    category_right=0
    number_right=0
    loc_right=0

    for i in predict['number'].keys():
        for j in ground_truth['number'].keys():
            if i==j:        ##标签对应上了
                category_right+=1
                if predict['number'][i]==ground_truth['number'][j]:     #统计数量一样，加1
                    number_right+=1
                ###计算位置对了几个
                loc_right+=count_common_elements_with_repetition(predict['location'][i],ground_truth['location'][j])

    rel_right=0        ###计算关系预测对了几个   逻辑上 关系三元组，一个个去比

  
    for i in predict['relationship']:
        for j in ground_truth['relationship']:
            if i[0]==j[0] and i[1]==j[1] and i[2]==j[2]:
                rel_right+=1     


    ### 计算ground_truth总数
    number_num=len(ground_truth['number'].keys())
    loc_num=sum(ground_truth['number'].values())
    rel_num=len(ground_truth['relationship'])

    ### 计算生成结果的总数
    gen_num=len(predict['number'].keys())
    gen_loc=len(predict['number'].values())
    gen_rel=len(predict['relationship'])

    ###计算 precision

    if gen_num==0:
        number_precision=0
        category_precision=0
    else:
        category_precision=category_right/gen_num
        number_precision=number_right/gen_num
    if gen_loc==0:
        loc_precision=0
    else:
        loc_precision=loc_right/gen_loc
    if gen_rel==0:
        rel_precision=0
    else:
        rel_precision=rel_right/gen_rel

    ### 计算recall
    if number_num==0:
        number_recall=1
        category_recall=1
    else:
        category_recall=category_right/number_num
        number_recall=number_right/number_num

    if loc_num==0:
        loc_recall=1
    else:
        loc_recall=loc_right/loc_num
    
    if rel_num==0:
        rel_recall=1
    else:
        rel_recall=rel_right/rel_num

    #### 计算F1
    if category_precision==0 and category_recall==0:
        category_f1=0
    else:
        category_f1=2*(category_precision*category_recall)/(category_precision+category_recall)
    if number_precision==0 and number_recall==0:
        num_f1=0
    else:
        num_f1=2*(number_precision*number_recall)/(number_precision+number_recall)
    
    if loc_precision==0 and loc_recall==0:
        loc_f1=0
    else:
        loc_f1=2*(loc_precision*loc_recall)/(loc_precision+loc_recall)
    
    if rel_precision==0 and rel_recall==0:
        rel_f1=0
    else:
        rel_f1=2*(rel_precision*rel_recall)/(rel_precision+rel_recall)

    return category_precision,number_precision,loc_precision,rel_precision,category_recall,number_recall,loc_recall,rel_recall,category_f1,num_f1,loc_f1,rel_f1



In [4]:
with open('VG_data\VG_ground_truth.pickle','rb') as f:
    ground_truth_total=pickle.load(f)

In [5]:
acc_list=[]

k=0

answer_list=os.listdir('VG_pre')

for i,answer in enumerate(answer_list[::-1]):

    with open(os.path.join('VG_pre',answer),'rb') as f:
        data=pickle.load(f)
        print(answer)
    
    ground_truth=ground_truth_total[int(answer[:-7])]

    if len(ground_truth['relationship'])==0:
        continue

    acc_list.append(list(compute_right_single(ground_truth,data)))

    k+=1
    if k==100:
        break



2339172.pickle
2339171.pickle
2339170.pickle
2339169.pickle
2339168.pickle
2339167.pickle
2339166.pickle
2339165.pickle
2339164.pickle
2339163.pickle
2339162.pickle
2339161.pickle
2339160.pickle
2339159.pickle
2339158.pickle
2339157.pickle
2339156.pickle
2339155.pickle
2339154.pickle
2339153.pickle
2339152.pickle
2339151.pickle
2339150.pickle
2339149.pickle
2339148.pickle
2339147.pickle
2339146.pickle
2339145.pickle
2339144.pickle
2339143.pickle
2339142.pickle
2339141.pickle
2339140.pickle
2339139.pickle
2339138.pickle
2339137.pickle
2339136.pickle
2339135.pickle
2339134.pickle
2339133.pickle
2339132.pickle
2339131.pickle
2339130.pickle
2339129.pickle
2339127.pickle
2339126.pickle
2339125.pickle
2339124.pickle
2339123.pickle
2339122.pickle
2339121.pickle
2339120.pickle
2339119.pickle
2339118.pickle
2339117.pickle
2339116.pickle
2339115.pickle
2339114.pickle
2339113.pickle
2339112.pickle
2339110.pickle
2339109.pickle
2339108.pickle
2339107.pickle
2339106.pickle
2339105.pickle
2339104.pi

In [147]:
acc_list

[[0.8571428571428571,
  0.5714285714285714,
  0.5714285714285714,
  0.0,
  0.375,
  0.25,
  0.23529411764705882,
  0.0,
  0.5217391304347825,
  0.34782608695652173,
  0.3333333333333333,
  0],
 [0.6666666666666666,
  0.6666666666666666,
  0.0,
  0.0,
  0.3333333333333333,
  0.3333333333333333,
  0.0,
  0.0,
  0.4444444444444444,
  0.4444444444444444,
  0,
  0],
 [0.25,
  0.0,
  0.25,
  0.0,
  0.6666666666666666,
  0.0,
  0.4,
  0.0,
  0.36363636363636365,
  0,
  0.3076923076923077,
  0],
 [0.45454545454545453,
  0.36363636363636365,
  0.2727272727272727,
  0.0,
  0.7142857142857143,
  0.5714285714285714,
  0.21428571428571427,
  0.0,
  0.5555555555555556,
  0.4444444444444444,
  0.23999999999999996,
  0],
 [0.3,
  0.3,
  0.1,
  0.011494252873563218,
  0.3333333333333333,
  0.3333333333333333,
  0.06666666666666667,
  0.125,
  0.3157894736842105,
  0.3157894736842105,
  0.08,
  0.02105263157894737],
 [0.23076923076923078,
  0.07692307692307693,
  0.0,
  0.0,
  0.3333333333333333,
  0.11

In [8]:
import numpy as np
np.set_printoptions(precision=4, suppress=True)
acc_numpy=np.array(acc_list)
print(acc_numpy.mean(axis=0))

[0.5135 0.3113 0.1936 0.032  0.5503 0.3468 0.1308 0.1172 0.4973 0.3056
 0.1443 0.0459]


In [7]:
(acc_numpy>0.55)[:,4:8].mean(1).mean()

0.195

In [138]:
acc_numpy[:,4:8]

array([[0.75  , 0.5   , 0.    , 0.    ],
       [0.3333, 0.    , 0.0769, 0.    ],
       [0.75  , 0.5   , 0.1667, 0.    ],
       [0.4   , 0.4   , 0.0769, 0.    ],
       [1.    , 1.    , 0.    , 1.    ],
       [0.7143, 0.4286, 0.    , 0.    ],
       [1.    , 1.    , 0.5   , 1.    ],
       [1.    , 0.    , 0.    , 1.    ],
       [0.5714, 0.4286, 0.    , 0.    ],
       [0.6667, 0.3333, 0.3333, 0.    ],
       [0.5   , 0.3333, 0.    , 0.    ],
       [0.7   , 0.4   , 0.0909, 0.    ],
       [0.75  , 0.75  , 0.    , 0.    ],
       [0.5714, 0.1429, 0.1333, 0.    ],
       [0.6667, 0.6667, 0.    , 0.    ],
       [0.5   , 0.5   , 0.    , 1.    ],
       [0.5556, 0.5556, 0.    , 1.    ],
       [0.6667, 0.    , 0.2   , 1.    ],
       [1.    , 1.    , 0.25  , 1.    ],
       [0.3636, 0.2727, 0.2727, 0.    ],
       [0.6364, 0.4545, 0.1176, 0.    ],
       [0.25  , 0.25  , 0.    , 0.    ],
       [1.    , 1.    , 0.    , 0.    ],
       [1.    , 1.    , 1.    , 1.    ],
       [0.75  , 

In [162]:
with open(os.path.join('VG_pre',answer_list[30]),'rb') as f:
    data=pickle.load(f)
    print(answer_list[30])
    
ground_truth=ground_truth_total[int(answer_list[30][:-7])]

2338998.pickle


In [163]:
data

{'number': {'chair': 2,
  'door': 1,
  'room': 1,
  'shelf': 1,
  'screen': 1,
  'wheel': 1,
  'paper': 1,
  'laptop': 1},
 'location': {'chair': ['top right corner', 'right'],
  'door': ['top right corner'],
  'room': ['right'],
  'shelf': ['top'],
  'screen': ['top left corner'],
  'wheel': ['center'],
  'paper': ['right'],
  'laptop': ['right']},
 'relationship': [['door', 'on', 'room'],
  ['shelf', 'behind', 'room'],
  ['screen', 'on', 'room'],
  ['chair', 'on', 'room'],
  ['laptop', 'near', 'room'],
  ['chair', 'on', 'room'],
  ['room', 'has', 'door'],
  ['paper', 'near', 'room'],
  ['wheel', 'behind', 'room'],
  ['room', 'near', 'shelf'],
  ['room', 'has', 'chair'],
  ['room', 'has', 'chair'],
  ['room', 'has', 'screen']]}

In [164]:
ground_truth

{'number': {'bike': 1, 'book': 6, 'chair': 2, 'room': 1, 'table': 1},
 'location': {'bike': ['lower left corner'],
  'book': ['top', 'lower', 'top left corner', 'top', 'top left corner', 'top'],
  'chair': ['center', 'right'],
  'room': ['top left corner'],
  'table': ['center']},
 'relationship': [['table', 'next to', 'chair']]}